In [25]:
import numpy as np
import os
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2
import NMO

In [35]:
class SeismicDataset(Dataset):
    """Seismic dataset."""

    def __init__(self, seismo_dir, velocity_dir, transform=None):
        """
        Args:
            seismo_dir (string): Directory with all the seismograms.
            velocity_dir (string): Directory with all the c1 velocity models.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        
        self.seismo_dir = seismo_dir
        self.velocity_dir = velocity_dir
        self.transform = transform

    def __len__(self):
        return len(os.listdir(root_dir + 'inputs/'))

    def __getitem__(self, idx):
        seismogram = np.loadtxt(self.seismo_dir + str(idx) + '.txt')[:, 1::2]
        velocity = np.fromfile(self.velocity_dir + str(idx) + '.bin', dtype='f')
        
        sample = {'seismogram': seismogram, 'velocity': velocity.reshape(250, 500)}

        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [36]:
class NormalMoveout(object):
    """Do normal moveout for seismogram."""

    def __init__(self, offsets, velocities, dt):
        self.offsets = offsets
        self.velocities = velocities
        self.dt = dt

    def __call__(self, sample):
        seismogram = sample['seismogram']
        corrected_seismogram = NMO.nmo_correction(seismogram, dt, offsets, velocities)

        return {'seismogram': corrected_seismogram, 'velocity': sample['velocity']}
    
class ToTensor(object):
    """Convert seismogram to tensor."""

    def __call__(self, sample):
        seismogram = sample['seismogram']
        tensor_seismogram = torch.from_numpy(seismogram)

        return {'seismogram': tensor_seismogram, 'velocity': sample['velocity']}

In [37]:
dt = 0.001

offsets = []
for i in range(50, 2050, 10):
    offsets.append(np.absolute(1250 - i))
    
velocities = []
for i in range(2000):
    velocities.append(1500)

nmo_transform = NormalMoveout(offsets=offsets, velocities=velocities, dt=dt)
dataset = SeismicDataset(seismo_dir='./train/raw/', 
                         velocity_dir='./train/outputs/', 
                         transform=transforms.Compose([ToTensor()]))

In [38]:
batch = dataset[1]

In [39]:
batch

{'seismogram': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 2.3626e-15,  1.6034e-15,  2.4988e-16,  ..., -7.3021e-15,
          -1.4063e-14, -1.5200e-14],
         [ 2.3627e-15,  1.4740e-15,  3.6686e-17,  ..., -8.4461e-15,
          -1.4462e-14, -1.4571e-14],
         [ 2.3272e-15,  1.3144e-15, -1.9387e-16,  ..., -9.4914e-15,
          -1.4796e-14, -1.3927e-14]], dtype=torch.float64),
 'velocity': array([[1500., 1500., 1500., ..., 1500., 1500., 1500.],
        [1500., 1500., 1500., ..., 1500., 1500., 1500.],
        [1500., 1500., 1500., ..., 1500., 1500., 1500.],
        ...,
        [2500., 2500., 2500., ..., 2500., 2500., 2500.],
        [2500., 2500., 2500., ..., 2500., 2500., 2500.],
        [

In [40]:
batch['seismogram'].shape

torch.Size([2000, 200])

In [41]:
batch['velocity'].shape

(250, 500)

In [42]:
import torch.nn as nn

In [ ]:
model = nn.Sequential(
    nn.Linear()
)